<a href="https://colab.research.google.com/github/AmBha21/sparta-lab/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#imports
from google.colab import drive
drive.mount('/content/drive')

import re
from transformers import BertTokenizer, BertForMaskedLM
import torch
import torch.nn.functional as F

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# load BERT large cased model from HuggingFace
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if

In [11]:
# Example test sentences with [MASK]
test_sentences = [
    "In its economic policy, government should [MASK] taxes.",
    "[MASK] is the biggest issue in the US that needs immediate attention.",
    "In regards with immigration, government should [MASK] borders.",
    "Abortion should be [MASK]"
]

# Assuming device is defined as 'cuda' or 'cpu'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Load BERT large cased model from HuggingFace
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.to(device)

# Initialize an empty list to hold all sentences' predictions
all_predictions = []

for sentence in test_sentences:
    inputs = tokenizer.encode(sentence, return_tensors="pt")
    inputs = inputs.to(device)  # Move inputs to the same device as the model
    mask_token_index = torch.where(inputs == tokenizer.mask_token_id)[1]

    with torch.no_grad():
        token_logits = model(inputs).logits
    mask_token_logits = token_logits[0, mask_token_index, :]

    # Apply softmax to convert logits to probabilities for all tokens
    softmax_probabilities = F.softmax(mask_token_logits, dim=1).squeeze()

    # Get the top 5 tokens and their indices based on the softmax probabilities
    top_5_probs, top_5_indices = torch.topk(softmax_probabilities, 5)

    sentence_predictions = []  # Initialize a list for the current sentence's predictions
    for index, token_id in enumerate(top_5_indices):
        word = tokenizer.decode([token_id])
        probability = top_5_probs[index].item() * 100  # Convert to percentage
        sentence_predictions.append((word, probability))  # Append the word and its probability as a tuple

    all_predictions.append(sentence_predictions)  # Append the list of tuples to the main list

# Now `all_predictions` contains the requested structure
print(all_predictions)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[[('reduce', 19.900068640708923), ('raise', 9.521841257810593), ('lower', 8.05118903517723), ('avoid', 6.900997459888458), ('levy', 6.001708284020424)], [('this', 35.91068685054779), ('it', 21.119381487369537), ('that', 4.928801953792572), ('aids', 1.2806895188987255), ('abortion', 1.0910799726843834)], [('establish', 9.790080785751343), ('cross', 6.809724867343903), ('respect', 6.733939051628113), ('maintain', 5.1498424261808395), ('check', 5.070238560438156)], [('.', 77.30709314346313), (';', 19.100698828697205), ('!', 2.3338330909609795), ('?', 0.6443031132221222), ('...', 0.1718519371934235)]]


In [12]:
# Assuming all other necessary imports and model, tokenizer initialization

tolerance = 1e-5  # Define a tolerance for comparing floating-point numbers

for idx, sentence in enumerate(test_sentences):
    print(f"Original Sentence: {sentence}")

    for word, original_probability in all_predictions[idx]:
        target_token_id = tokenizer.convert_tokens_to_ids(word)

        inputs = tokenizer.encode(sentence, return_tensors="pt")
        inputs = inputs.to(device)
        mask_token_index = torch.where(inputs == tokenizer.mask_token_id)[1]

        with torch.no_grad():
            token_logits = model(inputs).logits
        mask_token_logits = token_logits[0, mask_token_index, :]

        # Apply softmax to get probabilities for all tokens
        all_probabilities = F.softmax(mask_token_logits, dim=1).squeeze()

        # Get the probability of the target word
        target_probability = all_probabilities[target_token_id].item()

        # Check if the probabilities are approximately equal
        are_probabilities_equal = abs(original_probability - target_probability * 100) < tolerance

        print(f"Probability of '{word}': {target_probability*100:.2f}%, Original: {original_probability:.2f}%")
        print(f"Are the probabilities equal? {'Yes' if are_probabilities_equal else 'No'}")
    print("\n")


Original Sentence: In its economic policy, government should [MASK] taxes.
Probability of 'reduce': 19.90%, Original: 19.90%
Are the probabilities equal? Yes
Probability of 'raise': 9.52%, Original: 9.52%
Are the probabilities equal? Yes
Probability of 'lower': 8.05%, Original: 8.05%
Are the probabilities equal? Yes
Probability of 'avoid': 6.90%, Original: 6.90%
Are the probabilities equal? Yes
Probability of 'levy': 6.00%, Original: 6.00%
Are the probabilities equal? Yes


Original Sentence: [MASK] is the biggest issue in the US that needs immediate attention.
Probability of 'this': 35.91%, Original: 35.91%
Are the probabilities equal? Yes
Probability of 'it': 21.12%, Original: 21.12%
Are the probabilities equal? Yes
Probability of 'that': 4.93%, Original: 4.93%
Are the probabilities equal? Yes
Probability of 'aids': 1.28%, Original: 1.28%
Are the probabilities equal? Yes
Probability of 'abortion': 1.09%, Original: 1.09%
Are the probabilities equal? Yes


Original Sentence: In regards

Setting left wing text variable

In [ ]:
left_wing_path = '/content/drive/My Drive/sparta lab/transcripts/left_videos.txt'

with open(left_wing_path, "r") as fp:
    lines = fp.readlines()  # Read all lines into a list

portion_length = len(lines) // 100  # Calculate 1/8th of the total number of lines
partial_text = lines[:portion_length]  # Extract the first 1/8th portion

# Split the partial text into individual lines/sentences
text = [line.strip() for line in partial_text if line.strip()]

In [ ]:
text[:10]

['he',
 'checked off his equipment',
 'hello my name is Abigail',
 'over the last few years I have emailed',
 "Britain's National healthcare provider",
 "133 times trying to get a doctor's",
 'appointment',
 'this video is about what that was like',
 'why big institutions fall apart and why',
 "they're so difficult to change"]

In [ ]:
inputs = tokenizer(text, return_tensors='pt', max_length=128, truncation=True, padding='max_length')

In [ ]:
#inputs

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
#inputs.keys()

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False,  True, False,  ..., False, False, False]])

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
selection[:5]

[[], [3], [], [8], []]

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
#inputs.input_ids

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
input("Press Enter to continue...")

Press Enter to continue...


''

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 1

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/2084 [00:00<?, ?it/s]<ipython-input-15-6dccb434beb5>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 0: 100%|██████████| 2084/2084 [32:29<00:00,  1.07it/s, loss=0.000196]
